In [19]:
import numpy as np
import pandas as pd
import numpy.ma as ma
from scipy.optimize import curve_fit
import numpy.linalg as LA

#modulef = '/home/ubuntu/source/diff-classifier/diff_classifier/'
modulef = 'C:/Users/koolk/Desktop/diff-classifier/diff_classifier/'
import sys
sys.path.insert(0, modulef)

from utils import csv_to_pd
from msd import nth_diff, msd_calc, all_msds

In [2]:
d = {'Frame': [1, 2, 3, 4, 5],
             'X': [5, 6, 7, 8, 9],
             'Y': [6, 7, 8, 9, 10]}
df = pd.DataFrame(data=d)
df['MSDs'] = msd_calc(df)
df

,Frame,X,Y,MSDs
0,1,5,6,0.0
1,2,6,7,2.0
2,3,7,8,8.0
3,4,8,9,18.0
4,5,9,10,32.0


In [3]:
def alpha_calc(track):
    """
    Calculates the parameter alpha by fitting track MSD data to a function.
    
    Parameters
    ----------
    
    Returns
    ----------
    
    Examples
    ----------
    """

    x = track['Frame']
    y = track['MSDs']

    def msd_alpha(x, a, D):
        return 4*D*(x**a)

    try:
        popt, pcov = curve_fit(msd_alpha, x, y)
        a = popt[0]
        D = popt[1]
    except RuntimeError:
        print('Optimal parameters not found. Print NaN instead.')
        a = np.nan
        D = np.nan
    return a, D

In [42]:
def gyration_tensor(track):
    
    df = track
    
    Ta = np.sum((df['X'] - np.mean(df['X']))**2)/df['X'].shape[0]
    Tb = np.sum((df['Y'] - np.mean(df['Y']))**2)/df['Y'].shape[0]
    Tab = np.sum((df['X'] - np.mean(df['X']))*(df['X'] - np.mean(df['X'])))/df['X'].shape[0]

    w, v = LA.eig(np.array([[Ta, Tab], [Tab, Tb]]))
    l1 = max(w)
    l2 = min(w)
    return l1, l2

In [43]:
def asymmetry(track):

    l1, l2 = gyration_tensor(track)
    a1 = (l1**2 - l2**2)**2/(l1**2 + l2**2)**2
    a2 = l2/l1
    a3 = -np.log(1-((l1-l2)**2)/(2*(l1+l2)**2))

    return l1, l2, a1, a2, a3

In [45]:
l1, l2, a1, a2, a3 = asymmetry(df)

In [46]:
df['X']

0    5
1    6
2    7
3    8
4    9
Name: X, dtype: int64

In [52]:
df2 = df.as_matrix

In [56]:
type(df2)

method